In [1]:
import pandas as pd
import os
import json
from pandas.io.json import json_normalize
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
import torchvision
import torch.nn as nn
# from torchsummary import summary
import tokenization

import argparse
import collections
import logging
import json
import math
import os
import random
import six
from tqdm import tqdm_notebook as tqdm
from IPython.display import HTML, display

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import sys
import torchtext.data.dataset

import tokenization
from modeling import BertConfig, BertForMaskedLanguageModelling
from optimization import BERTAdam
from masked_language_model import notqdm, convert_tokens_to_features, LMProcessor, predict_masked_words, predict_next_words, improve_words_recursive

from sklearn.model_selection import train_test_split

import re
import csv

import sys
package_dir = "./"
sys.path.append(package_dir)

print(os.listdir())

['.circleci', '.git', '.github', '.gitignore', '.ipynb_checkpoints', 'askreddit_data.json', 'Bert ADL Project.ipynb', 'CONTRIBUTING.md', 'convert_tf_checkpoint_to_pytorch.py', 'data', 'dataset.py', 'detokenization.py', 'docker', 'docs', 'examples', 'file_0.csv', 'hubconf.py', 'hubconfs', 'LICENSE', 'MANIFEST.in', 'masked_language_model.py', 'modeling.py', 'notebooks', 'optimization.py', 'outputs.html', 'pytorch_pretrained_bert', 'README.md', 'requirements.txt', 'samples', 'setup.py', 'tests', 'test_threads.txt', 'tokenization.py', 'train_threads.txt', 'uncased_askreddit_data.tsv', '__init__.py', '__pycache__']


In [2]:
data = json.load(open('./askreddit_data.json', 'r'))

# ask_reddit_data =  pd.read_json('./askreddit_data.json')
# ask_reddit_data = json_normalize(data)
uncased_ask_reddit_data = pd.read_csv('./uncased_askreddit_data.tsv', sep='\t', header=0)

In [3]:
uncased_ask_reddit_data.head()
# uncased_ask_reddit_data['submission id'] == 'bn3bpz'

,submission score,comment score,submission id,comment id,title (string #1),comment (string#2)
0,80513,25495,bn3bpz,en27b8v,would you support a mandatory environmental se...,"not mandatory, but there was a voluntary progr..."
1,80513,4686,bn3bpz,en2anug,would you support a mandatory environmental se...,i feel like instead of forcing people to do so...
2,80513,2041,bn3bpz,en254lj,would you support a mandatory environmental se...,i wouldn't make it mandatory but maybe optiona...
3,80513,608,bn3bpz,en2gacy,would you support a mandatory environmental se...,"we already have americorps, where you serve fo..."
4,80513,310,bn3bpz,en2p5ro,would you support a mandatory environmental se...,i feel like forcing people to do it will just ...


In [4]:
array_id = uncased_ask_reddit_data['submission id']

set_of_ids = set(array_id)

print(set_of_ids, len(set_of_ids))

file_train, file_test = train_test_split(list(set_of_ids), random_state=42, test_size=0.10)
print(file_test, len(file_test))
# print(file_train, len(file_train))
# with open('train_threads.txt', 'a+') as file_train_ids:
#     for elem in file_train:
#         file_train_ids.write(elem + '\n')

# # with open('test_threads.txt', 'a+') as file_test_ids:
#     for elem in file_test:
#         file_test_ids.write(elem + '\n')

{'bqhfq9', 'bvr285', 'btexev', 'bojhpr', 'blrjnj', 'buz6lq', 'bkxl3u', 'boi2wy', 'bs3th0', 'bv51tj', 'buluna', 'br19bh', 'bl2scw', 'bltmmq', 'bkw3yd', 'bvgvkh', 'bty7a5', 'bmhq67', 'bkckiw', 'bro41c', 'btfu8k', 'blzj3m', 'bu1s5i', 'bk6w6e', 'bue7cv', 'bp9xm7', 'bnqk9j', 'bkjqjj', 'bpw8r2', 'bstmp7', 'bogywr', 'br87g0', 'bubm6g', 'bv6o78', 'bp182w', 'bvdaci', 'bo6854', 'bmlzqs', 'blf1l9', 'bvkru1', 'bmjk7w', 'bpr8xn', 'boxktx', 'bo3owh', 'bn3bpz', 'brzuwa', 'bskplh', 'bkkoq2', 'bndpre', 'bniexw', 'bkh4qb', 'bmac4g', 'bncqlb', 'bq7pbb', 'bu766u', 'bri3ja', 'blna2q', 'bobtwf', 'btb5si', 'bm8dfx', 'bmyv30', 'blcbrl', 'buso57', 'bqc3ib', 'bqg2xi', 'bmt72c', 'btm0yy', 'bls9ab', 'budsip', 'bljq8j', 'bkseqj', 'bugoyw', 'bk8ypo', 'bquclv', 'bsg50t', 'brayzq', 'bntob1', 'buunu0', 'bn3ab7', 'brlti4', 'bnlib2', 'bsomsy', 'bttd2p', 'bt3zc7', 'bqltzq', 'bqx0lw', 'bsahi8', 'bmhwr8', 'bovsdb', 'bq2sd5', 'bs6tqc'} 91
['bmjk7w', 'bu1s5i', 'bri3ja', 'bovsdb', 'bqhfq9', 'bnqk9j', 'bvkru1', 'btm0yy', 'bulu

In [20]:
# uncased_ask_reddit_data['submission id'] == 'bn3bpz'
# uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == 'bn3bpz']
train_df = [] #pd.DataFrame(columns=['submission score	comment score', 'submission id', 'comment id', 'title (string #1)', 'comment (string#2)'])
test_df = []

with open('train_threads.txt', 'r') as trt:
    for elem in trt.readlines():
#         print(elem.strip())
#         print(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
        print(elem)
        train_df.append(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
print(train_df)

bncqlb

bqg2xi

brzuwa

bobtwf

bri3ja

bq2sd5

bvkru1

bqltzq

blna2q

btm0yy

bkh4qb

bmt72c

boi2wy

bniexw

bv51tj

bnlib2

bp182w

brayzq

bmjk7w

blcbrl

boxktx

bvr285

bqx0lw

bq7pbb

bn3ab7

br87g0

bkjqjj

btb5si

buz6lq

bk6w6e

bntob1

bv6o78

bt3zc7

buluna

bp9xm7

btfu8k

bsahi8

bqhfq9

bo6854

bkxl3u

budsip

brlti4

bkseqj

bugoyw

bty7a5

bn3bpz

buunu0

bmhq67

bkckiw

bqc3ib

bsomsy

bnqk9j

bstmp7

bls9ab

bogywr

bquclv

bojhpr

blrjnj

bs3th0

bo3owh

bpw8r2

bmlzqs

blzj3m

bvgvkh

bpr8xn

buso57

btexev

bvdaci

br19bh

bu766u

blf1l9

bovsdb

bro41c

bs6tqc

bk8ypo

bl2scw

bndpre

bljq8j

bubm6g

bu1s5i

bmac4g
[      submission score  comment score submission id comment id  \
6280             29117           6431        bncqlb    en4pxcp   
6281             29117          24298        bncqlb    en4hp28   
6282             29117          15415        bncqlb    en4kqq9   
6283             29117          14389        bncqlb    en4f7vh   
6284             29117

[64 rows x 6 columns]]


In [33]:
with open('test_threads.txt', 'r') as tst:
    for elem in tst.readlines():
#         print(elem.strip())
#         print(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
        print(elem)
        test_df.append(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
print(test_df)

bmyv30

bsg50t

bm8dfx

bmhwr8

bkw3yd

bttd2p

bue7cv

bskplh

bltmmq

bkkoq2

[      submission score  comment score submission id comment id  \
2734             41748          17081        bmyv30    en1pqig   
2735             41748            732        bmyv30    en1fdkz   
2736             41748           2393        bmyv30    en20er1   
2737             41748           5848        bmyv30    en1nmnj   
2738             41748           4283        bmyv30    en1xgsh   
2739             41748            513        bmyv30    en1jx4b   
2740             41748           3336        bmyv30    en1sp35   
2741             41748           1177        bmyv30    en1wklc   
2742             41748           3035        bmyv30    en1lowf   
2743             41748            808        bmyv30    en1ylys   
2744             41748            203        bmyv30    en28uix   
2745             41748          14935        bmyv30    en1h2zi   
2746             41748          11955        bmyv30    en1q49

[90 rows x 6 columns]]


In [34]:
# print(train_df[:3])
# train_df_parsed = pd.DataFrame(train_df[0])
# train_df_parsed.to_csv(path_or_buf='./file_0.csv', sep='\t')

for i in range(len(train_df)):
    train_df_parsed = pd.DataFrame(train_df[i])
    train_df_parsed.to_csv(path_or_buf='./data/train/file_'+str(i)+'.csv', sep='\t')


In [35]:
for i in range(len(test_df)):
    test_df_parsed = pd.DataFrame(test_df[i])
    test_df_parsed.to_csv(path_or_buf='./data/test/file_'+str(i)+'.csv', sep='\t')


In [23]:
data_dict = dict()
data_dict['submission score'] = []
data_dict['comment score'] = []
data_dict['submission id'] = []
data_dict['comment id'] = []
data_dict['title (string #1)'] = []
data_dict['comment (string#2)'] = []

for i in range(len(train_df)):
#     print(i, './data/train/file_'+str(i)+'.csv')
    with open('./data/train/file_'+str(i)+'.csv') as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
    #     next(reader)
    #     df = pd.DataFrame(reader.data, columns=readr.columns)
    #     print(df.head())
        for row in reader:
    #         print(row)
    #         data_dict += row
            data_dict['submission score'].append(row['submission score'])
            data_dict['comment score'].append(row['comment score'])
            data_dict['submission id'].append(row['submission id'])
            data_dict['comment id'].append(row['comment id'])
            data_dict['title (string #1)'].append(row['title (string #1)'])
            data_dict['comment (string#2)'].append(row['comment (string#2)'])
        
# print(data_dict.values())

data_df_train = pd.DataFrame(data=data_dict)

print(data_df_train)

     submission score comment score submission id comment id  \
0               29117          6431        bncqlb    en4pxcp   
1               29117         24298        bncqlb    en4hp28   
2               29117         15415        bncqlb    en4kqq9   
3               29117         14389        bncqlb    en4f7vh   
4               29117          5357        bncqlb    en4m0gm   
5               29117         16081        bncqlb    en4hvq1   
6               29117         25806        bncqlb    en4n6hx   
7               29117          4892        bncqlb    en4izto   
8               29117         12290        bncqlb    en4hijc   
9               29117         17502        bncqlb    en4lisi   
10              29117         19126        bncqlb    en4et42   
11              29117         16508        bncqlb    en4je3s   
12              29117          4241        bncqlb    en4fcs2   
13              29117          6952        bncqlb    en4ip4o   
14              29117          2167     

[7204 rows x 6 columns]


In [24]:
# data_df_train.to_csv('./data/TrainFile.csv', index=False, sep='\t')
# data_df_train['comment score'].to_csv('./data/labels.csv', index=False, sep='\t')
data_df_train[['title (string #1)', 'comment (string#2)', 'comment score', 'submission score', 'submission id', 'comment id']].to_csv('./data/train.tsv', index=False, sep='\t')
# data_df_train['sep'] = '[SEP]'
data_df_train.head()

,submission score,comment score,submission id,comment id,title (string #1),comment (string#2)
0,29117,6431,bncqlb,en4pxcp,"if movie titles were taken literally, what blo...","mission impossible. ""sir, this mission is impo..."
1,29117,24298,bncqlb,en4hp28,"if movie titles were taken literally, what blo...",the silence of the lambs
2,29117,15415,bncqlb,en4kqq9,"if movie titles were taken literally, what blo...",the rock
3,29117,14389,bncqlb,en4f7vh,"if movie titles were taken literally, what blo...",50 shades of grey
4,29117,5357,bncqlb,en4m0gm,"if movie titles were taken literally, what blo...",meet joe black hi. (fin)


In [37]:
data_dict = dict()
data_dict['submission score'] = []
data_dict['comment score'] = []
data_dict['submission id'] = []
data_dict['comment id'] = []
data_dict['title (string #1)'] = []
data_dict['comment (string#2)'] = []

for i in range(len(test_df)):
#     print(i, './data/train/file_'+str(i)+'.csv')
    with open('./data/test/file_'+str(i)+'.csv') as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
    #     next(reader)
    #     df = pd.DataFrame(reader.data, columns=readr.columns)
    #     print(df.head())
        for row in reader:
    #         print(row)
    #         data_dict += row
            data_dict['submission score'].append(row['submission score'])
            data_dict['comment score'].append(row['comment score'])
            data_dict['submission id'].append(row['submission id'])
            data_dict['comment id'].append(row['comment id'])
            data_dict['title (string #1)'].append(row['title (string #1)'])
            data_dict['comment (string#2)'].append(row['comment (string#2)'])
            print(row)
        
# print(data_dict.values())

data_df_test = pd.DataFrame(data=data_dict)

# print(data_df_test)

OrderedDict([('', '2734'), ('submission score', '41748'), ('comment score', '17081'), ('submission id', 'bmyv30'), ('comment id', 'en1pqig'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)', 'kept finding small cut up straws in my grandmas bathroom. so many straws. i let my grandmother know that someone is using her bathroom to use drugs. she freaks out saying im accusing her of using drugs and that its impossible. a year later my uncle divorces his wife due to her cocaine addiction. grandma says well k always stopped by in the afternoon to use the restroom and then she would clean my whole house and i just didnt think anything of it! my favorite story at thanksgiving.')])
OrderedDict([('', '2735'), ('submission score', '41748'), ('comment score', '732'), ('submission id', 'bmyv30'), ('comment id', 'en1fdkz'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)

OrderedDict([('', '2739'), ('submission score', '41748'), ('comment score', '513'), ('submission id', 'bmyv30'), ('comment id', 'en1jx4b'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)', 'had a friend who started dating a guy while she was in the middle of rehab for alcohol and drug abuse. found out the guy was a "former meth user". i told her she was going to end up in prison or worse if she didn\'t break up with thim (she was on probation for multiple duis). within two months, not only was she in prison, she was pregnant while in prison.')])
OrderedDict([('', '2740'), ('submission score', '41748'), ('comment score', '3336'), ('submission id', 'bmyv30'), ('comment id', 'en1sp35'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)', 'the first avengers movie, my husband was wondering why the staff could stop the almighty tesseract. i casually said, "maybe, 

OrderedDict([('', '2751'), ('submission score', '41748'), ('comment score', '10835'), ('submission id', 'bmyv30'), ('comment id', 'en0xwji'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)', 'my wife was watching some tv show. i happened to walk in during some scene in which it was raining out, and a guy and a gal were looking at each other through their car windows as their kid went from one car to the other. i said "here comes the scene where they make up" without knowing any of the background of the show. my wife started telling me that it won\'t happen. they had had a failed relationship years earlier (hence the kid) and the woman was with a great guy. right after she said that, the woman got out and ran to the guy\'s car basically professing her love for him.')])
OrderedDict([('', '2752'), ('submission score', '41748'), ('comment score', '8776'), ('submission id', 'bmyv30'), ('comment id', 'en0rh1t'), ('title (strin

OrderedDict([('', '2761'), ('submission score', '41748'), ('comment score', '13048'), ('submission id', 'bmyv30'), ('comment id', 'en0yp70'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)', 'when a penny stock i bought around $0.03/share blew up to over $3. my only regret is not buying a lot more shares. edit: for those asking, here\'s my take based on my very limited experience with penny stocks. you can usually buy penny stock through your typical broker. you must be particularly wary of penny stocks because they can be subject to the phenomenon of "pump and dump": one or more people (likely those who have stake in whatever company is offering penny stock) hype it up and hope that people will buy. as the price creeps up, those stakeholders may decide to sell and thus reap the rewards, while those who naively assume going long is best will hold on to that stock with hopes that it\'ll blow up. the number of penny stock 

OrderedDict([('', '2769'), ('submission score', '41748'), ('comment score', '13904'), ('submission id', 'bmyv30'), ('comment id', 'en1ezkk'), ('title (string #1)', 'whats your greatest most satisfying "i fucking called it" moment?'), ('comment (string#2)', "my husband is super medically fragile - he's had cancer twice and a bone marrow transplant in the last 9 years. a few years ago he had surgery on his wrist and i had a gut feeling he was brewing an infection despite being on antibiotics. his surgeon's office saw him and switched abx. i contacted the cancer center because i just knew it was going to become more. they blew me off and punted back to the surgeon's office. i knew this was beyond the surgeon's scope. i pitched a tantrumy fit and pretty much told them they were going to see them and i wasn't accepting no for an answer. the triage phone nurse was condescending and telling me it was probably nothing and could wait. we got to the clinic and the nurse there started looking aro

...

OrderedDict([('', '2480'), ('submission score', '44253'), ('comment score', '374'), ('submission id', 'bkkoq2'), ('comment id', 'emhk2wb'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (string#2)', 'okay, another one. not me, this happened to my fiance recently. we live in new zealand, but both grew up in the usa. my fiance was talking to another american coworker about yosemite national park. one of his (kiwi) coworkers tells them, "it\'s pronounced yo-se-might". they\'re like no, it\'s yosemite. she wouldn\'t believe them.')])
OrderedDict([('', '2481'), ('submission score', '44253'), ('comment score', '2461'), ('submission id', 'bkkoq2'), ('comment id', 'emhg9q5'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (string#2)', 'everything. i am a creative director - clients always think they have better ideas and then waste my time

OrderedDict([('', '2489'), ('submission score', '44253'), ('comment score', '108'), ('submission id', 'bkkoq2'), ('comment id', 'emhsb7t'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (string#2)', 'im an application engineer for big plants and mills and such (power, paper, chemical, etc.) my specialization is to make sure that, given process data, their pumps, pipes, valves, and controllers (pneumatic, pressure-operated physical part) all work nicely with the dcs (digital control system, electric part) so nothing breaks or explodes a tank or whatnot. i had an old school professional engineer at a power company come to me and hand me a valve with a hole in it and basically tell me to fix it. let me explain, valves dont just magically get holes in them, thats a very strange mode of failure, and your newly-religious valves not going to get fixed, you have to get a new one at that point. this is about about a 

OrderedDict([('', '2498'), ('submission score', '44253'), ('comment score', '756'), ('submission id', 'bkkoq2'), ('comment id', 'emhvscw'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (string#2)', 'a drunk guy at a bar was trying to give me advice on how i should be performing and pursuing music. the manager had my back, and was trying to hint at the guy that im a professional and have had a good amount of experience. he didnt get the hint and he wouldnt stop, so the manager finally screamed listen to me you fucking idiot, she opened for reo speedwagon! she doesnt need your advice! manager got a good tip from me at the end of the night.')])
OrderedDict([('', '2499'), ('submission score', '44253'), ('comment score', '90'), ('submission id', 'bkkoq2'), ('comment id', 'emhvkvp'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (strin

OrderedDict([('', '2507'), ('submission score', '44253'), ('comment score', '72'), ('submission id', 'bkkoq2'), ('comment id', 'emi0jp9'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (string#2)', 'tl;dr: nascent coffee enthusiast guy tries to teach his barista roommate how to make coffee. not me but pretty close. i (male) live in an apartment with two other people one male, one female. the other guy tends to be act in lowkey sexist ways towards our female roommate, which is obvious to me because he will treat her differently even when we do the exact same things. anyway, this girl was a barista for three years. she knows how to make coffee. in the last month or so, the male roommate started getting really into coffee. he got a fancy grinder, fancy espresso machine, and started buying really good whole bean coffee. as his suitemates, he said we were welcome to use his equipment any time. one day, female roo

OrderedDict([('', '2513'), ('submission score', '44253'), ('comment score', '485'), ('submission id', 'bkkoq2'), ('comment id', 'emhknyg'), ('title (string #1)', 'whats the worst thing someone tried to correct you about something youre specialized at?'), ('comment (string#2)', 'i dont specialize in it, but i figured i would include it anyway since it pissed me off. ive been studying the korean language for years and years. last year, i moved to south korea to take a one-on-one accelerated course with a korean tutor. (i am hoping to someday adopt a korean child [not an infant] and move him/her to the u.s.; therefore, i want to make sure i can fully communicate.) because of this, i was pretty serious about paying the bigger bucks for a private tutor. with her, i studied for an average of 20 hours per week, plus an additional 10 or so on my own time. i dedicated a lot of time to my studies, in addition to practicing newly acquired skills with the locals. i would say i learned a lot. when 

In [42]:
# data_df_test.to_csv('./data/TestFile.csv', index=False, sep='\t')
# data_df_test['comment score'].to_csv('./data/labelsTest.csv', index=False, sep='\t')
data_df_test[['title (string #1)', 'comment (string#2)', 'comment score', 'submission score', 'submission id', 'comment id']].to_csv('./data/test.tsv', index=False, sep='\t')
# data_df_train['sep'] = '[SEP]'
data_df_test.shape[0] + data_df_train.shape[0]

8294

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

In [ ]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

In [ ]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
print(tokens_tensor)
print(segments_tensors)

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
if torch.cuda.is_available() == True:
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
print(model)
model.eval()

# If you have a GPU, put everything on cuda
if torch.cuda.is_available() == True:
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)
    print(predictions.size())
# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'
print(predicted_index)
print(predicted_token)

for (name, param) in model.named_parameters():
    if param.requires_grad:
        print('name', name#, 'weigth data', param.data
             )

In [ ]:
model = torchvision.models.vgg19(pretrained=True)
print(model)
for param in model.parameters():
    param.requires_grad = False
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(512, 8) # assuming that the fc7 layer has 512 neurons, otherwise change it 
# model.fc
# model.cuda()
# model.classifier[6] = nn.Linear(512, 8)
# model.fc
for (name, param) in model.named_parameters():
    if param.requires_grad:
        print('name', name, 'weigth data', param.data)